# Install libraries
import pandas as pd
import html5lib
!pip install lxml 

import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


In [204]:
# Create table from Wiki
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(wiki_url)

In [205]:
df = dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [206]:
# Clean up data. Remove codes where Borough is Not assigned. Save original 
# data, just in case we need it in the future. We also reset index and drop old 
# index to keep new dataframe clean.
new_df = df
new_df = new_df[df.Borough != 'Not assigned'].reset_index(drop=True)
new_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [207]:
# Change Neighbourhood to Borough name if Neighbourhood is Not assigned. For this,
# change Neighbourhood string to a list so that each item of the list can be checked
# and name of item can be changed if needed.
for rowNum in range(len(new_df)):
    neigh_list = new_df.iloc[rowNum]['Neighbourhood'].split(",")
    neigh_list = [item.strip() for item in neigh_list]
    new_df.iloc[rowNum]['Neighbourhood'] = neigh_list
    for neighNum in range(len(neigh_list)):
        if neigh_list[neighNum] == 'Not assigned':
            new_df.iloc[rowNum]['Neighbourhood'][neighNum] = new_df.iloc[rowNum]['Borough']

In [208]:
new_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]"
4,M7A,Downtown Toronto,"[Queen's Park, Ontario Provincial Government]"


In [209]:
new_df.shape

(103, 3)

In [210]:
# Now that dataframe is ready, let's merge neighbourhoods back so that they're
# together in string format. We will need it to be a simple string when we
# visualize the postal codes in the map
for rowNum in range(len(new_df)):
    new_df.iloc[rowNum]['Neighbourhood'] = ', '.join(new_df.iloc[rowNum]['Neighbourhood'])

new_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [211]:
# Geo data for latitude and longitude
!wget -q -O 'toronto_geo.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [212]:
# Create dataframe of postal codes with their latitudes and longitudes
toronto_geo_df = pd.read_csv('toronto_geo.csv')
toronto_geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [213]:
# Join the 2 dataframes to add latitude and longitude information for all
# Toronto Boroughs
toronto_df = new_df.join(toronto_geo_df.set_index('Postal Code'), on='Postal Code')
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [214]:
# Explore and cluster Downtown Toronto neighborhoods
dt_toronto_df = toronto_df.loc[toronto_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dt_toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [216]:
dt_toronto_df.shape

(19, 5)

In [217]:
# Get geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, Ontario'

geolocator = Nominatim(user_agent="dt_toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [218]:
dt_toronto_df['Neighbourhood']

0                             Regent Park, Harbourfront
1           Queen's Park, Ontario Provincial Government
2                              Garden District, Ryerson
3                                        St. James Town
4                                           Berczy Park
5                                    Central Bay Street
6                                              Christie
7                              Richmond, Adelaide, King
8     Harbourfront East, Union Station, Toronto Islands
9              Toronto Dominion Centre, Design Exchange
10                       Commerce Court, Victoria Hotel
11                       University of Toronto, Harbord
12            Kensington Market, Chinatown, Grange Park
13    CN Tower, King and Spadina, Railway Lands, Har...
14                                             Rosedale
15                                       Stn A PO Boxes
16                          St. James Town, Cabbagetown
17               First Canadian Place, Undergrou

In [219]:
# Let's visualize Downtown Toronto 
map_dt_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighbourhood in zip(dt_toronto_df['Latitude'],
                                 dt_toronto_df['Longitude'],
                                 dt_toronto_df['Neighbourhood']):
    label = folium.Popup(neighbourhood, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dt_toronto)

map_dt_toronto

In [220]:
# Let's explore 50 venues in each of the Postal Codes in Downtown Toronto
# Foursquare info
CLIENT_ID = '0W1D0ETZI5RMIRX0OTVR45XBEU0EPPABXKBU3DJS033SYYP4' # your Foursquare ID
CLIENT_SECRET = 'FLN4H0ZBHKIEYLXRONSAPRDLFTMBBUDJMAFZ2UMVFKNQCHOO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

# Define a function to find url
radius = 500
limit = 100
def getNearbyVenues(names, latitudes, longitudes):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            limit
            )

        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for
                                  item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude',
                             'Neighbourhood Longitude', 'Venue', 'Venue Latitude',
                             'Venue Longitude', 'Venue Category']
    return(nearby_venues)

In [221]:
# Call function
dt_toronto_venues = getNearbyVenues(names=dt_toronto_df['Neighbourhood'], 
                                   latitudes=dt_toronto_df['Latitude'],
                                   longitudes=dt_toronto_df['Longitude'])

In [222]:
# How many venues were returned for the different Downtown Toronto neighbourhoods
dt_toronto_venues.shape

(1245, 7)

In [223]:
# How many venues for each neighbourhood
dt_toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,64,64,64,64,64,64
Christie,16,16,16,16,16,16
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [224]:
# How many unique categories
print("There are {} unique categories.".format(len(dt_toronto_venues['Venue Category'].unique())))

There are 205 unique categories.


In [225]:
# Let's analyze each neighborhood
dt_toronto_onehot = pd.get_dummies(dt_toronto_venues[['Venue Category']], prefix="", prefix_sep="")
dt_toronto_onehot['Neighbourhood'] = dt_toronto_venues['Neighbourhood']
# Move Neighbourhood to first column
dt_toronto_onehot = dt_toronto_onehot[['Neighbourhood'] + [col for col in dt_toronto_onehot.columns
                                                         if col != 'Neighbourhood']]
dt_toronto_onehot.shape

(1245, 206)

In [226]:
# Group each neighbourhood and take mean of frequency of occurence of each category
dt_toronto_grouped = dt_toronto_onehot.groupby('Neighbourhood').mean().reset_index()
dt_toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.017241,0.0000,0.017241,0.034483,0.000000,0.000000,0.00,0.017241,0.017241,0.000000,0.034483,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.00,0.017241,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.034483,0.000000,0.0000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.086207,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.017241,0.000000,0.017241,0.00,0.000000,0.017241,0.00,0.000000,0.000000,0.017241,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.017241,0.017241,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.017241,0.017241,0.017241,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.017241,0.00,0.000000,0.017241,0.00,0.017241,0.017241,0.017241,0.017241,0.000000,0.000000,0.000000,0.00,0.00,0.0000

In [227]:
# Create dataframe with top 10 venues for each neighbourhood
columns = ['Neighbourhood']
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
for ind in range(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
dt_toronto_sorted = pd.DataFrame(columns=columns)
dt_toronto_sorted['Neighbourhood'] = dt_toronto_grouped['Neighbourhood']

# Function to find most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

for neigh in range(dt_toronto_grouped.shape[0]):
    dt_toronto_sorted.iloc[neigh,1:] = return_most_common_venues(dt_toronto_grouped.iloc[neigh,:], num_top_venues)

dt_toronto_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Beer Bar,Seafood Restaurant,Cocktail Bar,Farmers Market,Bakery,Cheese Shop,Restaurant,Café,Italian Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Coffee Shop,Boutique,Rental Car Location,Bar,Plane,Sculpture Garden
2,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Department Store,Salad Place,Burger Joint,Bubble Tea Shop,Bar
3,Christie,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Coffee Shop,Diner,Nightclub,Candy Store,Restaurant
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Pizza Place,Men's Store,Mediterranean Restaurant,Hotel
5,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Deli / Bodega,Japanese Restaurant,Italian Restaurant,Seafood Restaurant
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Steakhouse,American Restaurant,Seafood Restaurant,Asian Restaurant
7,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Italian Restaurant,Hotel,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Bookstore,Theater
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Sporting Goods Shop,Restaurant,Park
9,"Kensington Market, Chinatown, Grange Park",Café,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Bar,Dessert Shop,Dumpling Restaurant,Grocery Store,Burger Joint


In [228]:
# Apply K Means to cluster neighbourhood
kclusters = 5
dt_toronto_grouped_clustering = dt_toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_toronto_grouped_clustering)

In [229]:
# Add clustering labels
dt_toronto_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
dt_toronto_merged = dt_toronto_df
dt_toronto_merged = dt_toronto_merged.join(dt_toronto_sorted.set_index('Neighbourhood'), on='Neighbourhood')
dt_toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Performing Arts Venue
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,Sushi Restaurant,Sandwich Place,Distribution Center,Burrito Place,Café,Beer Bar,Smoothie Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Clothing Store,Coffee Shop,Café,Italian Restaurant,Hotel,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Bookstore,Theater
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Café,Coffee Shop,Clothing Store,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Breakfast Spot,Seafood Restaurant,Beer Bar
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Coffee Shop,Beer Bar,Seafood Restaurant,Cocktail Bar,Farmers Market,Bakery,Cheese Shop,Restaurant,Café,Italian Restaurant
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Department Store,Salad Place,Burger Joint,Bubble Tea Shop,Bar
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Coffee Shop,Diner,Nightclub,Candy Store,Restaurant
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,4,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Deli / Bodega,Thai Restaurant,Gym,Steakhouse,Concert Hall
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,4,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Sporting Goods Shop,Restaurant,Park
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,4,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Seafood Restaurant,Salad Place,Japanese Restaurant,Italian Restaurant,Sporting Goods Shop


In [232]:
# Visualize
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(dt_toronto_merged['Latitude'], dt_toronto_merged['Longitude'], dt_toronto_merged['Neighbourhood'], dt_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [235]:
# Examine each cluster
dt_toronto_merged.loc[dt_toronto_merged['Cluster Labels'] == 0, dt_toronto_merged.columns[[2] + list(range(5, dt_toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Diner,Yoga Studio,Sushi Restaurant,Sandwich Place,Distribution Center,Burrito Place,Café,Beer Bar,Smoothie Shop
5,Central Bay Street,0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Department Store,Salad Place,Burger Joint,Bubble Tea Shop,Bar


In [236]:
dt_toronto_merged.loc[dt_toronto_merged['Cluster Labels'] == 1, dt_toronto_merged.columns[[2] + list(range(5, dt_toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,1,Park,Playground,Trail,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [237]:
dt_toronto_merged.loc[dt_toronto_merged['Cluster Labels'] == 2, dt_toronto_merged.columns[[2] + list(range(5, dt_toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,2,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Coffee Shop,Diner,Nightclub,Candy Store,Restaurant


In [238]:
dt_toronto_merged.loc[dt_toronto_merged['Cluster Labels'] == 3, dt_toronto_merged.columns[[2] + list(range(5, dt_toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower, King and Spadina, Railway Lands, Har...",3,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Coffee Shop,Boutique,Rental Car Location,Bar,Plane,Sculpture Garden


In [241]:
dt_toronto_merged.loc[dt_toronto_merged['Cluster Labels'] == 4, dt_toronto_merged.columns[[2] + list(range(5, dt_toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",4,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Performing Arts Venue
2,"Garden District, Ryerson",4,Clothing Store,Coffee Shop,Café,Italian Restaurant,Hotel,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Bookstore,Theater
3,St. James Town,4,Café,Coffee Shop,Clothing Store,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Breakfast Spot,Seafood Restaurant,Beer Bar
4,Berczy Park,4,Coffee Shop,Beer Bar,Seafood Restaurant,Cocktail Bar,Farmers Market,Bakery,Cheese Shop,Restaurant,Café,Italian Restaurant
7,"Richmond, Adelaide, King",4,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Deli / Bodega,Thai Restaurant,Gym,Steakhouse,Concert Hall
8,"Harbourfront East, Union Station, Toronto Islands",4,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Sporting Goods Shop,Restaurant,Park
9,"Toronto Dominion Centre, Design Exchange",4,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Seafood Restaurant,Salad Place,Japanese Restaurant,Italian Restaurant,Sporting Goods Shop
10,"Commerce Court, Victoria Hotel",4,Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Deli / Bodega,Japanese Restaurant,Italian Restaurant,Seafood Restaurant
11,"University of Toronto, Harbord",4,Café,Restaurant,Bookstore,Bar,Japanese Restaurant,Bakery,Sandwich Place,Italian Restaurant,Beer Bar,Beer Store
12,"Kensington Market, Chinatown, Grange Park",4,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Bar,Dessert Shop,Dumpling Restaurant,Grocery Store,Burger Joint
